In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.pardir)))

In [2]:
import shutil

# Set up a clean test output directory
TEST_DIR = "test"
if os.path.exists(TEST_DIR):
    shutil.rmtree(TEST_DIR)
os.makedirs(TEST_DIR, exist_ok=True)

In [3]:
import pandas as pd
import numpy as np

# Define the scenario directory
SCENARIO_PATH = os.path.join(TEST_DIR, "scenario1")
os.makedirs(SCENARIO_PATH, exist_ok=True)

# Define the results directory
RESULTS_DIR = "results"
os.makedirs(RESULTS_DIR, exist_ok=True)

# Helper function to create and display a CSV file from a DataFrame
def create_csv(df, filename):
    path = os.path.join(SCENARIO_PATH, filename)
    df.to_csv(path, index=False)

# Use otoole to initialize the scenario structure
!otoole setup csv test/scenario1/ --overwrite

In [4]:
# User Settings
HOURS_PER_YEAR = 8760

# TIME
years = np.linspace(2026, 2027, num=1, dtype=int)
seasons = ['ALLSEASONS']
daytypes = ['ALLDAYS']
timebrackets = ['ALLTIMES']

# TOPOLOGY
regions = ['REGION1']

# DEMAND
demand = np.tile([100], len(years)) # MWh per year

# SUPPLY
technologies = ['GAS_CCGT', 'GAS_TURBINE']
residual_capacity = [0, 0] # MW
max_capacity = [1000/HOURS_PER_YEAR, 1000/HOURS_PER_YEAR] # MW
min_capacity = [0, 0] # MW

# PERFORMANCE
operating_life = [30, 25] # years
efficiency = [0.5, 0.4] # fraction of input energy converted to output energy
capacity_factors = [0.9, 0.8] # fraction of max capacity
availability = [1.0, 1.0] # fraction of year

# ECONOMICS
discount_rate = 0.05 
capital_costs = [500, 400] # $ / MW ($ per unit of extended capacity)
fixed_costs = [0, 0] # $ / MW / year ($ per unit of installed capacity per year)
variable_costs = [2, 5] # $ / MWh ($ per unit of activity)

# STORAGE (NOT IMPLEMENTED IN THIS EXAMPLE)

# EMISSIONS (NOT IMPLEMENTED IN THIS EXAMPLE)

# TARGETS (NOT IMPLEMENTED IN THIS EXAMPLE)

In [ ]:
# User Settings
HOURS_PER_YEAR = 8760

# TIME - Multi-year with varying granularity
years = np.array([2025, 2030, 2035, 2040, 2045, 2050])
seasons = ['WINTER', 'SPRING', 'SUMMER', 'FALL']
daytypes = ['WEEKDAY', 'WEEKEND']
timebrackets = ['MORNING', 'AFTERNOON', 'EVENING', 'NIGHT']

# TOPOLOGY
regions = ['REGION1']

# DEMAND - Non-uniform growth pattern with seasonal variation
base_demand = 1000  # MWh per year
demand = np.array([
    base_demand * (1.03 ** (y - 2026)) * (1 + 0.1 * np.sin(i * np.pi / len(years)))
    for i, y in enumerate(years)
])  # Compound growth with oscillation

# SUPPLY - Diverse technology portfolio
technologies = [
    'GAS_CCGT',      # Base load, high capital, low variable
    'GAS_TURBINE',   # Peaker, low capital, high variable
    'COAL',          # Long-lived, mid-cost
    'SOLAR_PV',      # Zero variable cost, time-dependent
    'WIND',          # Zero variable cost, time-dependent
    'NUCLEAR'        # Very high capital, very low variable, very long life
]

residual_capacity = [50, 0, 100, 0, 0, 0]  # MW - existing capacity
max_capacity = [500, 300, 400, 800, 600, 200]  # MW
min_capacity = [0, 0, 0, 0, 0, 0]  # MW

# PERFORMANCE - Varying characteristics
operating_life = [30, 20, 40, 25, 25, 60]  # years - spans different lengths
efficiency = [0.58, 0.35, 0.38, 1.0, 1.0, 0.33]  # fraction
capacity_factors = [0.85, 0.90, 0.80, 0.25, 0.35, 0.90]  # Mean values (will be time-varying for solar/wind)
availability = [0.95, 0.98, 0.85, 1.0, 0.95, 0.90]  # fraction

# ECONOMICS - Wide cost spectrum
discount_rate = 0.07  # Higher discount rate amplifies salvage value differences
capital_costs = [800, 400, 1200, 1000, 1500, 5000]  # $ / MW - wide range
fixed_costs = [25, 10, 40, 20, 30, 100]  # $ / MW / year
variable_costs = [35, 80, 25, 0, 0, 5]  # $ / MWh - includes zero marginal cost

# STORAGE (NOT IMPLEMENTED IN THIS EXAMPLE)

# EMISSIONS (NOT IMPLEMENTED IN THIS EXAMPLE)

# TARGETS (NOT IMPLEMENTED IN THIS EXAMPLE)

In [6]:
# YEAR
year_df = pd.DataFrame({'VALUE': years})
create_csv(year_df, 'YEAR.csv')

# REGION
region_df = pd.DataFrame({'VALUE': regions})
create_csv(region_df, 'REGION.csv')

# TECHNOLOGY
tech_df = pd.DataFrame({'VALUE': technologies})
create_csv(tech_df, 'TECHNOLOGY.csv')

# FUEL
fuel_df = pd.DataFrame({'VALUE': ['ELEC', 'GAS']})
create_csv(fuel_df, 'FUEL.csv')

# MODE_OF_OPERATION
mode_df = pd.DataFrame({'VALUE': [1]})
create_csv(mode_df, 'MODE_OF_OPERATION.csv')

# SEASON
season_df = pd.DataFrame({'VALUE': seasons})
create_csv(season_df, 'SEASON.csv')

# DAYTYPE
daytype_df = pd.DataFrame({'VALUE': daytypes})
create_csv(daytype_df, 'DAYTYPE.csv')

# DAILYTIMEBRACKET
dailytimebracket_df = pd.DataFrame({'VALUE': timebrackets})
create_csv(dailytimebracket_df, 'DAILYTIMEBRACKET.csv')

# TIMESLICE
timeslices = [f"{s}_{d}_{t}" for s in seasons for d in daytypes for t in timebrackets]
timeslice_df = pd.DataFrame({'VALUE': timeslices})
create_csv(timeslice_df, 'TIMESLICE.csv')

In [7]:
# SpecifiedAnnualDemand
demand_df = pd.DataFrame({
    'REGION': regions * len(years),
    'FUEL': ['ELEC'] * len(years),
    'YEAR': years,
    'VALUE': demand
})
create_csv(demand_df, 'SpecifiedAnnualDemand.csv')

# SpecifiedDemandProfile
demand_profile_df = pd.DataFrame({
    'REGION': np.repeat(regions, len(years)*len(timeslices)),
    'FUEL': ['ELEC'] * len(years) * len(timeslices),
    'TIMESLICE': np.tile(timeslices, len(years)),
    'YEAR': np.repeat(years, len(timeslices)),
    'VALUE': [1.0/len(timeslices)] * (len(years) * len(timeslices))
    # Proportion (%) of annual demand
    # CURRENTLY EVENLY DISTRIBUTED ACROSS TIMESLICES
})
create_csv(demand_profile_df, 'SpecifiedDemandProfile.csv')

# CapacityFactor
capacity_factor_df = pd.DataFrame({
    'REGION': np.repeat(regions, len(technologies)*len(years)*len(timeslices)),
    'TECHNOLOGY': np.repeat(technologies, len(years)*len(timeslices)),
    'TIMESLICE': np.tile(timeslices, len(technologies) * len(years)),
    'YEAR': np.tile(np.repeat(years, len(timeslices)), len(technologies)),
    'VALUE': np.repeat(capacity_factors, len(years)*len(timeslices))
    # Capacity factors (%) for each technology across all timeslices and years
    # CURRENTLY ONE VALUE PER TECHNOLOGY
})
create_csv(capacity_factor_df, 'CapacityFactor.csv')

# AvailabilityFactor
availability_factor_df = pd.DataFrame({
    'REGION': np.repeat(regions, len(technologies)*len(years)),
    'TECHNOLOGY': np.repeat(technologies, len(years)),
    'YEAR': np.array([years] * len(technologies)).flatten(),
    'VALUE': np.repeat(availability, len(years))
    # Availability factors (%) for each technology and year
    # CURRENTLY ONE VALUE PER TECHNOLOGY
})
create_csv(availability_factor_df, 'AvailabilityFactor.csv')

# CapitalCost
capital_cost_df = pd.DataFrame({
    'REGION': np.repeat(regions, len(technologies)*len(years)),
    'TECHNOLOGY': np.repeat(technologies, len(years)),
    'YEAR': np.array([years] * len(technologies)).flatten(),
    'VALUE': np.repeat(capital_costs, len(years))
    # Capital cost [$ per MWh]
    # CURRENTLY ONE VALUE PER TECHNOLOGY
})
create_csv(capital_cost_df, 'CapitalCost.csv')

# FixedCost
fixed_cost_df = pd.DataFrame({
    'REGION': np.repeat(regions, len(technologies)*len(years)),
    'TECHNOLOGY': np.repeat(technologies, len(years)),
    'YEAR': np.array([years] * len(technologies)).flatten(),
    'VALUE': np.repeat(fixed_costs, len(years))
    # Fixed cost [$ per MWh per year]
    # CURRENTLY ONE VALUE PER TECHNOLOGY
})
create_csv(fixed_cost_df, 'FixedCost.csv')

# CapacityToActivityUnit
capacity_to_activity_df = pd.DataFrame({
    'REGION': np.repeat(regions, len(technologies)),
    'TECHNOLOGY': technologies,
    'VALUE': [HOURS_PER_YEAR] * len(technologies)
    # Energy that would be produced when one unit of capacity is fully used in one year
    # (MWh / capacity unit) * 8760 hours per year
    # CURRENTLY ONE VALUE PER TECHNOLOGY
    # (this explicitly sets 1 capacity unit = 1 MW, 1 activity unit = 1 hour at full capacity)
})
create_csv(capacity_to_activity_df, 'CapacityToActivityUnit.csv')

# VariableCost
variable_cost_df = pd.DataFrame({
    'REGION': np.repeat(regions, len(technologies)*len(years)),
    'TECHNOLOGY': np.repeat(technologies, len(years)),
    'MODE_OF_OPERATION': [1] * len(technologies)*len(years),
    'YEAR': np.array([years] * len(technologies)).flatten(),
    'VALUE': np.repeat(variable_costs, len(years))
    # Variable cost [$ per activity unit]
    # (since 1 activity unit = 1 hour at full capacity, this is $ / hour)
    # CURRENTLY ONE VALUE PER TECHNOLOGY
})
create_csv(variable_cost_df, 'VariableCost.csv')

# InputActivityRatio
input_ratio_df = pd.DataFrame({
    'REGION': np.repeat(regions, len(technologies)*len(years)),
    'TECHNOLOGY': np.repeat(technologies, len(years)),
    'FUEL': np.repeat(['GAS'], len(technologies)*len(years)),
    'MODE_OF_OPERATION': [1] * len(technologies)*len(years),
    'YEAR': np.array([years] * len(technologies)).flatten(),
    'VALUE': np.repeat([1.0]/np.array(efficiency), len(years))
    # MWh fuel / MWh electricity
    # CURRENTLY ONE VALUE PER TECHNOLOGY
})
create_csv(input_ratio_df, 'InputActivityRatio.csv')

# OutputActivityRatio
output_ratio_df = pd.DataFrame({
    'REGION': np.repeat(regions, len(technologies)*len(years)),
    'TECHNOLOGY': np.repeat(technologies, len(years)),
    'FUEL': np.repeat(['ELEC'], len(technologies)*len(years)),
    'MODE_OF_OPERATION': [1] * len(technologies)*len(years),
    'YEAR': np.array([years] * len(technologies)).flatten(),
    'VALUE': np.repeat([1.0]*len(technologies), len(years))
    # MWh electricity / MWh electricity
    # CURRENTLY ONE VALUE PER TECHNOLOGY
})
create_csv(output_ratio_df, 'OutputActivityRatio.csv')

In [8]:
# Conversionls - maps timeslices to seasons
conv_ls_rows = []
for ts in timeslices:
    for s in seasons:
        conv_ls_rows.append({
            'TIMESLICE': ts,
            'SEASON': s,
            'VALUE': 1 if ts.startswith(s) else 0
        })
conv_ls_df = pd.DataFrame(conv_ls_rows)
create_csv(conv_ls_df, 'Conversionls.csv')

# Conversionld - maps timeslices to daytypes
conv_ld_rows = []
for ts in timeslices:
    for d in daytypes:
        conv_ld_rows.append({
            'TIMESLICE': ts,
            'DAYTYPE': d,
            'VALUE': 1 if f"_{d}_" in ts else 0
        })
conv_ld_df = pd.DataFrame(conv_ld_rows)
create_csv(conv_ld_df, 'Conversionld.csv')

# Conversionlh - maps timeslices to dailytimebrackets
conv_lh_rows = []
for ts in timeslices:
    for h in timebrackets:
        conv_lh_rows.append({
            'TIMESLICE': ts,
            'DAILYTIMEBRACKET': h,
            'VALUE': 1 if ts.endswith(h) else 0
        })
conv_lh_df = pd.DataFrame(conv_lh_rows)
create_csv(conv_lh_df, 'Conversionlh.csv')

# DaysInDayType - days per season/daytype/year combination
days_in_day_type_df = pd.DataFrame({
    'SEASON': np.tile(np.repeat(seasons, len(daytypes)), len(years)),
    'DAYTYPE': np.tile(daytypes, len(seasons) * len(years)),
    'YEAR': np.repeat(years, len(seasons) * len(daytypes)),
    'VALUE': [365 / len(daytypes) / len(seasons)] * (len(years) * len(seasons) * len(daytypes))
    # LEAP YEARS?
    # np.array([365] * len(years)) + (years % 4 == 0)
})
create_csv(days_in_day_type_df, 'DaysInDayType.csv')

# DaySplit - fraction of year for each timebracket per year
day_split_df = pd.DataFrame({
    'DAILYTIMEBRACKET': np.tile(timebrackets, len(years)),
    'YEAR': np.repeat(years, len(timebrackets)),
    'VALUE': [1 / (len(timebrackets) * 365)] * (len(years) * len(timebrackets))
    # Length of one timebracket in one specific day as a fraction of the year
})
create_csv(day_split_df, 'DaySplit.csv')

# YearSplit - fraction of year for each timeslice per year
year_split_df = pd.DataFrame({
    'TIMESLICE': np.tile(timeslices, len(years)),
    'YEAR': np.repeat(years, len(timeslices)),
    'VALUE': [1 / len(timeslices)] * (len(years) * len(timeslices))
    # Duration of a modelled timeslice as a fraction of the year
})
create_csv(year_split_df, 'YearSplit.csv')


In [9]:
# OperationalLife
op_life_df = pd.DataFrame({
    'REGION': regions * len(technologies),
    'TECHNOLOGY': technologies,
    'VALUE': operating_life
})
create_csv(op_life_df, 'OperationalLife.csv')

# ResidualCapacity
residual_cap_df = pd.DataFrame({
    'REGION': regions * len(technologies)*len(years),
    'TECHNOLOGY': np.repeat(technologies, len(years)),
    'YEAR': np.array([years] * len(technologies)).flatten(),
    'VALUE': np.repeat(residual_capacity, len(years))
})
create_csv(residual_cap_df, 'ResidualCapacity.csv')

# TotalAnnualMaxCapacity
total_max_cap_df = pd.DataFrame({
    'REGION': regions * len(technologies) * len(years),
    'TECHNOLOGY': np.repeat(technologies, len(years)),
    'YEAR': np.array([years] * len(technologies)).flatten(),
    'VALUE': np.repeat(max_capacity, len(years))
})
create_csv(total_max_cap_df, 'TotalAnnualMaxCapacity.csv')

# TotalAnnualMinCapacity
total_min_cap_df = pd.DataFrame({
    'REGION': regions * len(technologies)*len(years),
    'TECHNOLOGY': np.repeat(technologies, len(years)),
    'YEAR': np.array([years] * len(technologies)).flatten(),
    'VALUE': np.repeat(min_capacity, len(years))
})
create_csv(total_min_cap_df, 'TotalAnnualMinCapacity.csv')

# Discount Rate
discount = pd.DataFrame({
    'REGION': regions,
    'VALUE': [discount_rate] * len(regions)
    # CURRENTLY ONE VALUE PER REGION
})
create_csv(discount, 'DiscountRate.csv')

In [10]:
!otoole convert csv datafile test/scenario1 test/scenario1.txt ../docs/OSeMOSYS_config.yaml

In [11]:
!glpsol -m ../docs/OSeMOSYS.txt -d test/scenario1.txt --wglp test/scenario1.glp --write test/scenario1.sol

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 -m ../docs/OSeMOSYS.txt -d test/scenario1.txt --wglp test/scenario1.glp --write
 test/scenario1.sol
Reading model section from ../docs/OSeMOSYS.txt...
1425 lines were read
Reading data section from test/scenario1.txt...
2157 lines were read
Checking Max and Min capcity-investment bounds for r in REGION, t in TECHNOLOGY, y in YEAR 
Checking (line 175)...
Checking Annual activity limits for r in REGION, t in TECHNOLOGY, y in YEAR 
Checking (line 180)...
Checking Residual and TotalAnnualMax Capacity for r in REGION, t in TECHNOLOGY, y in YEAR 
Checking (line 185)...
Checking Residual, Total annual maxcap and mincap investments for  all Region, Tech and Year 
Checking (line 190)...
Checking Annual production by technology bounds for r in REGION, t in TECHNOLOGY, y in YEAR 
Checking (line 195)...
Checking TimeSlices/YearSplits for y in YEAR 
Checking (line 200)...
Checking (line 201)...
Checking Model period activit

In [12]:
!otoole results glpk csv test/scenario1.sol results datafile test/scenario1.txt ../docs/OSeMOSYS_config.yaml --glpk_model test/scenario1.glp

In [13]:
# Create the equivalent PyPSA network
import pypsa
from pypsa.common import annuity

# Initialize the network
n = pypsa.Network()

# Add carriers
n.add("Carrier", "GAS")
n.add("Carrier", "AC") # ELEC

# Add the buses
n.add("Bus", region_df['VALUE'].tolist(), carrier="AC")

# Set investment periods
periods = year_df['VALUE'].tolist()
n.set_investment_periods(periods)

# Set investment period weightings
# Calculate the duration each period represents
period_durations = np.diff(years, prepend=years[0])  # Years from start for each period
investment_period_weightings = pd.DataFrame(index=years)
investment_period_weightings['years'] = period_durations
investment_period_weightings['objective'] = [
    1 / ((1 + discount_rate) ** (y - years[0])) for y in years
]
n.investment_period_weightings = investment_period_weightings

# Set snapshots
timesteps = timeslice_df['VALUE'].tolist()
snapshots = pd.MultiIndex.from_product([periods, timesteps], names=['period', 'timestep'])
n.set_snapshots(snapshots)

# Set snapshot weightings
year_split_df_temp = year_split_df.copy()
year_split_df_temp['period'] = year_split_df_temp['YEAR']
year_split_df_temp['timestep'] = year_split_df_temp['TIMESLICE']
hours_per_snapshot = year_split_df_temp.set_index(['period', 'timestep'])['VALUE'] * HOURS_PER_YEAR
hours_per_snapshot = hours_per_snapshot.reindex(snapshots)
n.snapshot_weightings['objective'] = hours_per_snapshot
n.snapshot_weightings['generators'] = hours_per_snapshot

# Set up demand
demand_pypsa = pd.merge(
    demand_df,
    demand_profile_df,
    on=['REGION', 'FUEL', 'YEAR'],
    suffixes=('_annual', '_profile')
)
# Energy per snapshot = Annual Demand × Demand Profile
demand_pypsa['energy_MWh'] = demand_pypsa['VALUE_annual'] * demand_pypsa['VALUE_profile']
demand_pypsa['period'] = demand_pypsa['YEAR']
demand_pypsa['timestep'] = demand_pypsa['TIMESLICE']

# Power = Energy / Hours
energy_series = demand_pypsa.set_index(['period', 'timestep'])['energy_MWh']
power = energy_series.divide(hours_per_snapshot).reindex(snapshots)

n.add("Load",
      "demand",
      bus="REGION1",
      p_set=power)  # MW

# Add generators
p_max_pu = pd.DataFrame(index=snapshots)

for idx, tech in enumerate(technologies):
    # Calculate annualized capital cost (matching OSeMOSYS CapitalCost + FixedCost)
    annualized_capital_cost = capital_costs[idx] * annuity(operating_life[idx], discount_rate) + fixed_costs[idx]
    
    # For PyPSA multi-investment periods, capital_cost should be annualized
    n.add("Generator",
          tech,
          bus="REGION1",
          carrier="GAS",
          p_nom=residual_capacity[idx],  # Existing capacity (MW)
          p_nom_max=max_capacity[idx],   # Maximum total capacity (MW)
          p_nom_min=min_capacity[idx],   # Minimum total capacity (MW)
          p_nom_extendable=True,         # Allow capacity expansion
          capital_cost=annualized_capital_cost,  # Annualized capital + fixed cost ($/MW/year)
          marginal_cost=variable_costs[idx],     # Variable cost ($/MWh)
          efficiency=efficiency[idx],            # Conversion efficiency
          lifetime=operating_life[idx],          # Asset lifetime (years)
          build_year=years[0]                    # First period when new capacity can be built
          )
    
    # Set capacity factor × availability factor for time-dependent availability
    # In OSeMOSYS: TotalActivityUpperLimit = TotalCapacity × CapacityFactor × AvailabilityFactor
    p_max_pu[tech] = capacity_factors[idx] * availability[idx]

n.generators_t.p_max_pu = p_max_pu

# Print network components to verify
print("--- PyPSA Network Components ---")
print("\nGenerators:\n", n.generators[['p_nom', 'p_nom_extendable', 'p_nom_max', 'capital_cost', 'marginal_cost', 'efficiency', 'lifetime']])
print("\nInvestment Period Weightings:\n", n.investment_period_weightings)
print("\nTime-varying load (first 5 snapshots):\n", n.loads_t.p_set.head())
print("\nTime-varying generator availability (first 5 snapshots):\n", n.generators_t.p_max_pu.head())
print("\nSnapshot Weightings (first 5):\n", n.snapshot_weightings.head())

INFO:pypsa.network.index:Repeating time-series for each investment period and converting snapshots to a pandas.MultiIndex.


--- PyPSA Network Components ---

Generators:
              p_nom  p_nom_extendable  p_nom_max  capital_cost  marginal_cost  \
name                                                                           
GAS_CCGT      50.0              True      500.0  1.123476e+05           35.0   
GAS_TURBINE    0.0              True      300.0  4.169011e+04           80.0   
COAL         100.0              True      400.0  2.097295e+05           25.0   
SOLAR_PV       0.0              True      800.0  1.226118e+05            0.0   
WIND           0.0              True      600.0  1.839176e+05            0.0   
NUCLEAR        0.0              True      200.0  1.199815e+06            5.0   

             efficiency  lifetime  
name                               
GAS_CCGT           0.58      30.0  
GAS_TURBINE        0.35      20.0  
COAL               0.38      40.0  
SOLAR_PV           1.00      25.0  
WIND               1.00      25.0  
NUCLEAR            0.33      60.0  

Investment Period Weigh

In [14]:
# Run the PyPSA optimization
n.optimize(solver_name='glpk', compute_infeasibilities=True, multi_investment_periods=True)

INFO:linopy.model: Solve problem using Glpk solver
INFO:linopy.io: Writing time: 0.01s
INFO:linopy.solvers:GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --lp /var/folders/ny/csqtllsd23b8m43n1kthbfhm0000gp/T/linopy-problem-l1zepf01.lp
 --output /var/folders/ny/csqtllsd23b8m43n1kthbfhm0000gp/T/linopy-solve-vx71o7go.sol
Reading problem data from '/var/folders/ny/csqtllsd23b8m43n1kthbfhm0000gp/T/linopy-problem-l1zepf01.lp'...
2252 rows, 1031 columns, 4108 non-zeros
12614 lines were read
GLPK Simplex Optimizer 5.0
2252 rows, 1031 columns, 4108 non-zeros
Preprocessing...
1216 rows, 1030 columns, 3072 non-zeros
Scaling...
 A: min|aij| =  2.500e-01  max|aij| =  1.000e+00  ratio =  4.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 1216
      0: obj =  -8.047232495e+07 inf =   3.388e+01 (192)
      6: obj =  -7.903807803e+07 inf =   8.604e-16 (0)
*   711: obj =  -8.023089322e+07 inf =   0.000e+00 (0) 1
OPTIMAL LP 

('ok', 'optimal')

In [15]:
# Load and Display OSeMOSYS Results
print("\n--- OSeMOSYS Optimization Results ---")
osemosys_objective = pd.read_csv(os.path.join(RESULTS_DIR, 'TotalDiscountedCost.csv'))
print("\nObjective:", osemosys_objective)
osemosys_total_capacity = pd.read_csv(os.path.join(RESULTS_DIR, 'TotalCapacityAnnual.csv'))
print("\nOptimal Capacities (p_nom_opt):\n", osemosys_total_capacity)
osemosys_total_production = pd.read_csv(os.path.join(RESULTS_DIR, 'TotalTechnologyAnnualActivity.csv'))
print("\nTotal Production:\n", osemosys_total_production)


--- OSeMOSYS Optimization Results ---

Objective:     REGION  YEAR        VALUE
0  REGION1  2025  5527.259529
1  REGION1  2030  3689.948643
2  REGION1  2035  2630.826810
3  REGION1  2040  1871.076564
4  REGION1  2045  1327.574485
5  REGION1  2050   946.700291

Optimal Capacities (p_nom_opt):
      REGION   TECHNOLOGY  YEAR         VALUE
0   REGION1     GAS_CCGT  2025  5.000000e+01
1   REGION1     GAS_CCGT  2030  5.000000e+01
2   REGION1     GAS_CCGT  2035  5.000000e+01
3   REGION1     GAS_CCGT  2040  5.000000e+01
4   REGION1     GAS_CCGT  2045  5.000000e+01
5   REGION1     GAS_CCGT  2050  5.000000e+01
6   REGION1  GAS_TURBINE  2025  0.000000e+00
7   REGION1  GAS_TURBINE  2030  0.000000e+00
8   REGION1  GAS_TURBINE  2035  0.000000e+00
9   REGION1  GAS_TURBINE  2040  0.000000e+00
10  REGION1  GAS_TURBINE  2045  0.000000e+00
11  REGION1  GAS_TURBINE  2050  0.000000e+00
12  REGION1         COAL  2025  1.000000e+02
13  REGION1         COAL  2030  1.000000e+02
14  REGION1         COAL  2035

In [16]:
# Display PyPSA Results
print("\n--- PyPSA Optimization Results ---")
print("\nObjective:", n.objective)
print("\nOptimal Capacities (p_nom_opt):\n", n.generators.p_nom_opt)
print("\nTotal Production (p * HOURS_PER_YEAR):\n", n.generators_t.p * HOURS_PER_YEAR)


--- PyPSA Optimization Results ---

Objective: -80230893.22

Optimal Capacities (p_nom_opt):
 name
GAS_CCGT       0.301742
GAS_TURBINE    0.000000
COAL           0.000000
SOLAR_PV       0.000000
WIND           0.000000
NUCLEAR        0.000000
Name: p_nom_opt, dtype: float64

Total Production (p * HOURS_PER_YEAR):
 name                               GAS_CCGT  GAS_TURBINE  COAL  SOLAR_PV  \
period timestep                                                            
2025   WINTER_WEEKDAY_MORNING     970.87080          0.0   0.0       0.0   
       WINTER_WEEKDAY_AFTERNOON   970.87080          0.0   0.0       0.0   
       WINTER_WEEKDAY_EVENING     970.87080          0.0   0.0       0.0   
       WINTER_WEEKDAY_NIGHT       970.87080          0.0   0.0       0.0   
       WINTER_WEEKEND_MORNING     970.87080          0.0   0.0       0.0   
...                                     ...          ...   ...       ...   
2050   FALL_WEEKDAY_NIGHT        2134.43532          0.0   0.0       0.0   

OSeMOSYS: NPV with Salvage Value

OSeMOSYS computes Net Present Value of all costs, crediting back the residual value of assets that outlive the model horizon.

Inputs:
- NewCapacity = 0.012684 MW
- CapitalCost = 500 $/MW
- VariableCost = 2 $/MWh
- OperationalLife = 30 years
- DiscountRate = 0.05
- Model Period = 1 year (2026)

1. Capital Investment:
   - CapitalInvestment = 500 × 0.012684 = $6.34
   - DiscountedCapitalInvestment = $6.34 / (1.05)^0 = $6.34

2. Salvage Value (asset life extends beyond model):
   - Since (2026 + 30 - 1) = 2055 > 2026, salvage applies.
   
   - SV = 6.34 × (1 - [(1.05)^1 - 1] / [(1.05)^30 - 1])
      = 6.34 × (1 - 0.05 / 3.322)
      = 6.34 × 0.985
      = $6.24
   
   - DiscountedSalvageValue = 6.24 / (1.05)^1 = $5.94

3. Operating Cost:
   - OperatingCost = 100 MWh × 2 $/MWh = $200
   - DiscountedOperatingCost = 200 / (1.05)^0.5 = $195.18 (mid-year)

4. Total Discounted Cost:
   = DiscountedCapital + DiscountedOperating - DiscountedSalvage
   = 6.34 + 195.18 - 5.94
   = $195.58

PyPSA: Multi-investment Period Cost Accounting

When using `multi_investment_periods=True` with total (non-annualized) capital costs, PyPSA computes costs as follows:

Inputs:
- NewCapacity = 0.012684 MW
- capital_cost = 500 $/MW (total investment, NOT annualized)
- marginal_cost = 2 $/MWh
- Production = 100 MWh
- investment_period_weightings['objective'] = 1.0 (for 2026)

1. Capital Investment:
   - CapitalCost × Capacity × InvestmentPeriodWeight
   = 500 × 0.012684 × 1.0
   = $6.34

2. Operating Cost:
   - MarginalCost × Energy × InvestmentPeriodWeight
   = 2 × 100 × 1.0
   = $200.00

3. Total:
   = $6.34 + $200.00
   = $206.34


Reconciling the Difference
| Component | PyPSA | OSeMOSYS | Difference |
| --------- | ----- | -------- | ---------- |
| Capital Investment | $6.34 | $6.34 | $0.00 |
| Salvage Value Credit | $0.00 | −$5.94 | +$5.94 |
| Operating Cost | $200.00 | $195.18 | +$4.82 |
| Total | $206.34 | $195.58 | +$10.76 |

In [17]:
n.statistics()

Optimal Capacity                                               \
                          2025     2030     2035     2040     2045     2050   
Generator GAS          0.30174  0.30174  0.30174  0.30174  0.30174  0.30174   
Load      -            0.00000  0.00000  0.00000  0.00000  0.00000  0.00000   

              Installed Capacity                       ...      Revenue  \
                            2025   2030   2035   2040  ...         2035   
Generator GAS              150.0  150.0  150.0  150.0  ...  49621.96674   
Load      -                  0.0    0.0    0.0    0.0  ... -49621.92810   

                                                      Market Value             \
                      2040         2045          2050         2025       2030   
Generator GAS  58234.75252  66687.72510  631542.64088         35.0  34.999259   
Load      -   -58234.77273 -66687.82351 -631542.19462          NaN        NaN   

                                                           
                    2035       2040      2045        2050  
Generator GAS  34.999135  34.999474  34.99954  295.879135  
Load      -          NaN        NaN       NaN         NaN  

[2 rows x 67 columns]

PyPSA: Curtailment

| Metric | Value |
| ------ | ----- |
| Optimal Capacity | 0.012684 MW |
| Input p_max_pu (Capacity Factor limit) | 0.9 |
| Hours | 8760 |
| Maximum Available Generation | 0.012684 × 0.9 × 8760 = 100.008 MWh |
| Actual Supply	| 99.99978 MWh |
| Curtailment | 0.00009 MWh |
| Load | 100.0 MWh |

Curtailment is defined as:
Curtailment = Available Generation − Actual Generation
            = p_nom_opt × p_max_pu × hours - Demand

- Curtailment is an artifact of floating-point arithmetic precision, LP solver convergence tolerance
- Need to find an $\epsilon$ to flag large differences